In [1]:
import xarray as xr
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import csv
from geopy.distance import geodesic
from datetime import timedelta
import pickle
from scipy.spatial import cKDTree

In [2]:
tracking_start, tracking_end = "2012-01-01", "2012-12-31"

experiment_name = 'INALT20r.L120-KRS006'#'INALT60.L120-KRS0020' #
data_resolution = '1d'

OW_thr_factor =-0.3
first_or_last_crossing = 'first'
minimum_duration = 5

In [3]:
if experiment_name.startswith("INALT60"):
    prefix = "2_"
    Npix_min, Npix_max = 720, 18000
    sigma = 15
    wx = 600
    ds_name = "inalt60"
    OW_end = 20120125
elif experiment_name.startswith("INALT20"):
    prefix = "1_"
    Npix_min, Npix_max = 80, 2000
    sigma = 5
    wx = 200
    ds_name = "inalt20r"
    OW_end = 20120219

In [4]:
#depth= 0  #corresponding to... 
#depth_index = 0 

mesh_mask = xr.open_dataset(f'/gxfs_work/geomar/smomw523/smoothed_data/{experiment_name}/{prefix}{experiment_name}_mesh_mask.nc')
nav_lat = mesh_mask['nav_lat'].values  # shape (y, x)
nav_lon = mesh_mask['nav_lon'].values

indices = np.concatenate((range(0, 11, 10),range(18, 25, 6),range(29, 34, 4),range(36, 40, 3),range(41, 120, 2)))
depth_information = [(round(mesh_mask.nav_lev.values[i]), i) for i in indices]
print(len(depth_information),depth_information)

48 [(0, 0), (21, 10), (54, 18), (91, 24), (132, 29), (173, 33), (210, 36), (253, 39), (286, 41), (323, 43), (365, 45), (412, 47), (465, 49), (525, 51), (592, 53), (668, 55), (753, 57), (848, 59), (952, 61), (1066, 63), (1189, 65), (1321, 67), (1461, 69), (1608, 71), (1762, 73), (1922, 75), (2086, 77), (2255, 79), (2428, 81), (2603, 83), (2782, 85), (2963, 87), (3146, 89), (3331, 91), (3518, 93), (3706, 95), (3896, 97), (4086, 99), (4279, 101), (4472, 103), (4666, 105), (4861, 107), (5057, 109), (5254, 111), (5452, 113), (5651, 115), (5850, 117), (6050, 119)]


In [5]:
ds_vel = xr.open_mfdataset(f'/gxfs_work/geomar/smomw523/processed_data/{experiment_name}/velocity/{ds_name}-{data_resolution}-velocity-*.nc').vel
OW_exp = xr.open_dataset(f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-0/OW_20120101_{OW_end}_rolling-{wx}.nc').OW.isel(time=0)

In [13]:
min_diameter, min_duration = 50, 10
filename = f'Tracks-crossed_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle'

with open(f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/Flierl-criterion_for-crossed-eddies-more-than-{min_diameter}km-{min_duration}days_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['track_ix', 'type', 'lon', 'lat', 'time', 'duration',
                     'depth', 'rotational_velocity', 'translational_velocity', 'Flierl_criterion',
                     'amp', 'scale', 'area', 'section'])
        
    for (depth, depth_index) in tqdm(depth_information): # depth_information
        datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-{depth}/'
        filepath = datapath + filename
        if not os.path.exists(filepath):
            print(f"File not found: {filepath}")
            continue
        tracks = np.load(datapath + filename, allow_pickle=True)
        df = pd.read_csv(datapath + f'Good-hope-{first_or_last_crossing}-crossings_min-{minimum_duration}d_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.csv')

        for track_ix, track in enumerate(tracks):
            if track['scale'].max() < min_diameter or len(track['time']) < min_duration:
                continue
            #else:
            speeds = [0]
            times, lons, lats = track['time'], track['lon'], track['lat']
            for i in range(1, len(times)):
                dist_km = geodesic((lats[i-1], lons[i-1]), (lats[i], lons[i])).m
                speed = dist_km / 86400
                speeds.append(speed)
            track['speed'] = np.array(speeds)

            if df.loc[track_ix, 'section']!='Ansorge-GHS':
                continue # crossing wrong section

            target_time = pd.to_datetime(df.loc[track_ix, 'time']) + timedelta(hours=12)
            matching_index = (times == target_time).nonzero()[0]
            if len(matching_index) != 1:
                if len(matching_index) == 0:
                    print(f"Warning: Track crosses section at first detected timestep (no velocity to calculate). Skip. ({depth}, {track_ix})")
                elif len(matching_index) == None:
                    print(f"Warning: Times from track and crossing.csv do not match. Skip. ({depth}, {track_ix})")
                elif len(matching_index) > 1: # should not be possible
                    print(f"Warning: More than one matching timestep. Skip. ({depth}, {track_ix})") 
                continue
            else:
                translat_vel = track['speed'][matching_index]

            eddies = np.load(datapath + f'Eddies_{target_time.strftime("%Y-%m-%d")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle', allow_pickle=True)

            rot_vel = None
            for eddy_ix in range(len(eddies)):
                eddy = eddies[eddy_ix]
                if np.abs(eddy['lon'][0] - df.loc[track_ix, 'lon']) < 1e-3 and np.abs(eddy['lat'][0] - df.loc[track_ix, 'lat']) < 1e-3:
                    ds_vel_slice = ds_vel.isel(depth=depth_index).sel(time_counter=target_time)

                    eddy_lat = OW_exp['lat'].values[eddy['eddy_j']]
                    eddy_lon = OW_exp['lon'].values[eddy['eddy_i']]
                    eddy_coords = np.column_stack((eddy_lat, eddy_lon))
                    flat_latlon = np.column_stack((nav_lat.ravel(), nav_lon.ravel()))
                    tree = cKDTree(flat_latlon)
                    _, indices = tree.query(eddy_coords)
                    ys, xs = np.unravel_index(indices, nav_lat.shape)
                    mask = np.zeros(ds_vel_slice.shape, dtype=bool)
                    mask[ys, xs] = True
                    masked_vel = ds_vel_slice.where(mask) 
                    if masked_vel.count() == 0:
                        print(f"Warning: No mask for eddy. Skip. ({depth}, {track_ix}, {eddy_ix})")
                        continue # should not be possible
                    rot_vel = masked_vel.fillna(0).values.max()
                    Flierl_vel = rot_vel / translat_vel
                    writer.writerow([track_ix, track['type'], df.loc[track_ix, 'lon'], df.loc[track_ix, 'lat'], df.loc[track_ix, 'time'], df.loc[track_ix, 'duration'],
                        depth, rot_vel, translat_vel[0], Flierl_vel[0],
	                    df.loc[track_ix, 'amp'], df.loc[track_ix, 'scale'], df.loc[track_ix, 'area'], df.loc[track_ix, 'section']])
                    break
                else:
                    continue
            if rot_vel is None:
                print(f"Warning: No eddy found along track. Skip. ({depth}, {track_ix})") # should not be possible
                continue

        with open(filepath, "wb") as f:
            pickle.dump(tracks, f)    

100%|██████████| 48/48 [14:36<00:00, 18.27s/it]

File not found: /gxfs_work/geomar/smomw523/eddytools/results/INALT60.L120-KRS0020/smoothed/15/1d/depth-6050/Tracks-crossed_20120101_20121231_OW0.3_Npix-720-18000_rolling-600.pickle


In [6]:
# surface eddy as column downward
 
min_diameter, min_duration = 50, 10
filename = f'Tracks-crossed_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle'

with open(f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/Flierl-criterion-column_for-crossed-eddies-more-than-{min_diameter}km-{min_duration}days_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['track_ix', 'type', 'lon', 'lat', 'time', 'duration',
                     'depth', 'rotational_velocity', 'translational_velocity', 'Flierl_criterion',
                     'amp', 'scale', 'area', 'section'])
    
    datapath = f'/gxfs_work/geomar/smomw523/eddytools/results/{experiment_name}/smoothed/{sigma}/{data_resolution}/depth-0/' #surface
    filepath = datapath + filename
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
    
    tracks = np.load(datapath + filename, allow_pickle=True)
    
    df = pd.read_csv(datapath + f'Good-hope-{first_or_last_crossing}-crossings_min-{minimum_duration}d_{tracking_start.replace("-", "")}_{tracking_end.replace("-", "")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.csv')
    print(len(tracks))
    for track_ix, track in tqdm(enumerate(tracks)):
        if track['scale'].max() < min_diameter or len(track['time']) < min_duration:
            continue
            #else:
        speeds = [0]
        times, lons, lats = track['time'], track['lon'], track['lat']
        for i in range(1, len(times)):
            dist_km = geodesic((lats[i-1], lons[i-1]), (lats[i], lons[i])).m
            speed = dist_km / 86400
            speeds.append(speed)
        track['speed'] = np.array(speeds)

        if df.loc[track_ix, 'section']!='Ansorge-GHS':
            continue # crossing wrong section

        target_time = pd.to_datetime(df.loc[track_ix, 'time']) + timedelta(hours=12)
        matching_index = (times == target_time).nonzero()[0]
        if len(matching_index) != 1:
            if len(matching_index) == 0:
                print(f"Warning: Track crosses section at first detected timestep (no velocity to calculate). Skip. ({depth}, {track_ix})")
            elif len(matching_index) == None:
                print(f"Warning: Times from track and crossing.csv do not match. Skip. ({depth}, {track_ix})")
            elif len(matching_index) > 1: # should not be possible
                print(f"Warning: More than one matching timestep. Skip. ({depth}, {track_ix})") 
            continue
        else:
            translat_vel = track['speed'][matching_index]

        eddies = np.load(datapath + f'Eddies_{target_time.strftime("%Y-%m-%d")}_OW{np.abs(OW_thr_factor)}_Npix-{Npix_min}-{Npix_max}_rolling-{wx}.pickle', allow_pickle=True) #surface
        for (depth, depth_index) in depth_information:
            rot_vel = None
            for eddy_ix in range(len(eddies)):
                eddy = eddies[eddy_ix]
                if np.abs(eddy['lon'][0] - df.loc[track_ix, 'lon']) < 1e-3 and np.abs(eddy['lat'][0] - df.loc[track_ix, 'lat']) < 1e-3:
                    ds_vel_slice = ds_vel.isel(depth=depth_index).sel(time_counter=target_time)

                    eddy_lat = OW_exp['lat'].values[eddy['eddy_j']]
                    eddy_lon = OW_exp['lon'].values[eddy['eddy_i']]
                    eddy_coords = np.column_stack((eddy_lat, eddy_lon))
                    flat_latlon = np.column_stack((nav_lat.ravel(), nav_lon.ravel()))
                    tree = cKDTree(flat_latlon)
                    _, indices = tree.query(eddy_coords)
                    ys, xs = np.unravel_index(indices, nav_lat.shape)
                    mask = np.zeros(ds_vel_slice.shape, dtype=bool)
                    mask[ys, xs] = True
                    masked_vel = ds_vel_slice.where(mask) 
                    if masked_vel.count() == 0:
                        print(f"Warning: No mask for eddy. Skip. ({depth}, {track_ix}, {eddy_ix})")
                        continue # should not be possible
                    rot_vel = masked_vel.fillna(0).values.max()
                    Flierl_vel = rot_vel / translat_vel
                    writer.writerow([track_ix, track['type'], df.loc[track_ix, 'lon'], df.loc[track_ix, 'lat'], df.loc[track_ix, 'time'], df.loc[track_ix, 'duration'],
                        depth, rot_vel, translat_vel[0], Flierl_vel[0],
	                    df.loc[track_ix, 'amp'], df.loc[track_ix, 'scale'], df.loc[track_ix, 'area'], df.loc[track_ix, 'section']])
                    break
                else:
                    continue
            if rot_vel is None:
                print(f"Warning: No eddy found along track. Skip. ({depth}, {track_ix})") # should not be possible
                continue

        with open(filepath, "wb") as f:
            pickle.dump(tracks, f)    

52


52it [04:50,  5.60s/it]
